### Callin Switzer

20 Oct 2017

-Read in all the Kalmia polygon images (from R KalmiaVisualizePolygons_V2.R)

-Stack images

-Visualize a heatmap

-Save heatmap

In [1]:
import cv2

import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

In [2]:
# %qtconsole

In [3]:
# load colormap
os.chdir("/Users/cswitzer/Dropbox/ExperSummer2016/Kalmia/Manuscript/DatasetsSupplemental/KalmiaDigitizedPolygon/colormap")
import matplotlib.cm as cm
import colormaps as cmaps
colormap_r = matplotlib.colors.ListedColormap(cmaps.magma.colors[::-1])

In [7]:
os.chdir("/Users/cswitzer/Dropbox/ExperSummer2016/Kalmia/KalmiaExamples/KalmiaManualTrig_VidsToProcess")

In [8]:
# list files in directory
mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
photoList = [ x for x in onlyfiles if not x.startswith('.') and x.endswith('.png')]

In [9]:
len(photoList) # should be 29

29

In [87]:
# stack all polygons and visualize 
stackedImg = np.ones((2000, 2000)) * 0.0

for ii in range(len(photoList)):
    # read in image
    
    fpth = photoList[ii]
    img = cv2.imread(fpth,0)
    img = (img < 255) * 1.0
    if ii == 0:
        img = cv2.flip(img, 1)
    img2 = mask * img
    stackedImg = img2 + stackedImg

plt.clf()
io.imshow(stackedImg, cmap = 'YlOrRd')

In [12]:
# convert 0's to NAN
stackedImg = stackedImg / np.max(stackedImg)
stackedImg[stackedImg == 0.0] = np.nan

In [13]:
# remove border
a = stackedImg
slice_val = 150
b = a[slice_val:-slice_val, slice_val:-slice_val]

In [14]:
plt.close("all")


levels =  np.arange(0, 1.2, 0.1)
fig, ax = plt.subplots()
ax.axis('equal')
ax.axis('off')
#ax.tick_params('off')
CF = ax.imshow(b, cmap = colormap_r)
cbar = fig.colorbar(CF, ticks = levels, fraction = 0.031, pad = 0)
#cbar.ax.set_yticklabels(["0", "2", "3", "4"])
plt.tight_layout()

fig.savefig("Figures/RawStacked_29Polygons.png", dpi = 700)



In [15]:
# resample stacked images 
def resampleImgsNoWeight():
    # stack all polygons and visualize
    stackedImg = np.ones((2000, 2000)) * 0.0

    # make multiplication matrix
    mask = np.ones((2000, 2000))

    photoSamp = np.random.choice(range(len(photoList)), size=len(photoList), replace=True, p=None)


    for ii in photoSamp:
        # read in imag
        fpth = photoList[ii]
        img = cv2.imread(fpth,0)
        img = (img < 255) * 1.0



        #flip the one that I did backwards
        if fpth == '20160615_121016.png':
            img = cv2.flip(img, 1)

        img2 = mask * img

        # stack images
        stackedImg = img2 + stackedImg
    
    return stackedImg

In [18]:
stt = time.time()
imStt = np.array([resampleImgsNoWeight() for ii in range(100)])
print time.time() - stt

307.041757107


In [41]:
# sum up arrays to make total bootstrap resample 
imSum = imStt.sum(axis = 0)
i2 = imSum

# normalize
imSum = imSum / np.max(imSum)

plt.clf()
io.imshow(imSum, cmap = colormap_r)
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])

In [42]:
#imSum[imSum == 0] = np.nan

imSum = np.ma.masked_where(imSum == 0, imSum)

In [43]:
levels =  np.arange(0, 1.2, 0.1)
fig, ax = plt.subplots()
ax.axis('equal')
ax.axis('off')
#ax.tick_params('off')
CF = ax.imshow(imSum, cmap = colormap_r, interpolation='none')
cbar = fig.colorbar(CF, ticks = levels, fraction = 0.031, pad = 0)
ax.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.tight_layout()

fig.savefig("Figures/RawStacked_29Polygons_withPoints.png", dpi = 700)

In [44]:
a = imSum
slice_val = 150
c = a[slice_val:-slice_val, slice_val:-slice_val]

In [45]:
levels =  np.arange(0, 1.2, 0.1)
fig, ax = plt.subplots()
ax.axis('equal')
ax.axis('off')
#ax.tick_params('off')
CF = ax.imshow(c, cmap = colormap_r, interpolation='none')
cbar = fig.colorbar(CF, ticks = levels, fraction = 0.031, pad = 0)
plt.tight_layout()

fig.savefig("Figures/RawStacked_29Polygons_NoPoints.png", dpi = 700)

In [46]:
plt.close("all")

In [47]:
os.path.abspath(photoList[kk])

'/Users/cswitzer/Dropbox/ExperSummer2016/Kalmia/KalmiaExamples/KalmiaManualTrig_VidsToProcess/20160615_121016.png'

In [48]:
# add  blur
plt.clf()
kernel = np.ones((50,50),np.float32)/25
imSum2 = np.array(cv2.filter2D(i2.copy(),-1,kernel))

# normalize
imSum2 = imSum2 / np.max(imSum2) * 100


plt.close('all')


imNA = imSum2
imNA[imNA <= 10] = np.nan
fig, ax = plt.subplots()
im = ax.imshow(imNA, cmap = colormap_r, interpolation='none')
cax = fig.add_axes([0.78, 0.2, 0.05, 0.6])
fig.colorbar(im, cax=cax, orientation='vertical')

# add contour lines
ax.contour(imSum2, levels = range(10, 100, 20), colors = 'k')
# plt.xlim([600, 1400])
# plt.ylim([ 1700, 700])

ax.axis('off')

ax.set_xlim([600, 1900])
ax.set_ylim([ 1800, 700])

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
ax.scatter(x = intPoints[:,0], y = intPoints[:,1])

plt.show()

plt.savefig('Figures/SingleAntherHeatmap_magmaCmap_blurr_withPoints.png', dpi = 500)

In [49]:
# add  blur
plt.clf()
kernel = np.ones((50,50),np.float32)/25
imSum2 = np.array(cv2.filter2D(i2.copy(),-1,kernel))

# normalize
imSum2 = imSum2 / np.max(imSum2) * 100


plt.close('all')


imNA = imSum2
imNA[imNA <= 10] = np.nan
fig, ax = plt.subplots()
im = ax.imshow(imNA, cmap = colormap_r, interpolation='none')
cax = fig.add_axes([0.78, 0.2, 0.05, 0.6])
fig.colorbar(im, cax=cax, orientation='vertical')

# add contour lines
ax.contour(imSum2, levels = range(10, 100, 20), colors = 'k')
# plt.xlim([600, 1400])
# plt.ylim([ 1700, 700])

ax.axis('off')

ax.set_xlim([600, 1900])
ax.set_ylim([ 1800, 700])

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
#ax.scatter(x = intPoints[:,0], y = intPoints[:,1])

plt.show()

plt.savefig('Figures/SingleAntherHeatmap_magmaCmap_blurr_NoPoints.png', dpi = 500)

In [79]:
plt.clf()
kernel = np.ones((1,1),np.float32)/25
imSum2 = np.array(cv2.filter2D(i2.copy(),-1,kernel))

# normalize
imSum2 = imSum2 / np.max(imSum2) * 100


plt.close('all')


imNA = imSum2
imNA[imNA <= 10] = np.nan
fig, ax = plt.subplots()
im = ax.imshow(imNA, cmap = colormap_r, interpolation='none')
cax = fig.add_axes([0.78, 0.2, 0.05, 0.6])
cbar = fig.colorbar(im, cax=cax, ticks = range(10, 100, 20),  orientation='vertical')

# add contour lines
ax.contour(imSum2, levels = [1, 20, 40,60], colors = 'k')
cbar.ax.set_yticklabels([str(ii / 100.0) for ii in range(10, 100, 20)])
# plt.xlim([600, 1400])
# plt.ylim([ 1700, 700])

ax.axis('off')

ax.set_xlim([600, 1900])
ax.set_ylim([ 1800, 700])

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
#ax.scatter(x = intPoints[:,0], y = intPoints[:,1])

plt.show()

plt.savefig('Figures/SingleAntherHeatmap_magmaCmap_blurr_NoPoints_little.png', dpi = 500)

In [103]:
plt.clf()

imSum2 = imSum

# normalize
imSum2 = imSum2 / np.max(imSum2) * 100


plt.close('all')


imNA = imSum2
imNA[imNA <= 10] = np.nan
fig, ax = plt.subplots()
im = ax.imshow(imNA, cmap = colormap_r, interpolation='none')
cax = fig.add_axes([0.78, 0.2, 0.05, 0.6])
cbar = fig.colorbar(im, cax=cax, ticks = range(10, 100, 20),  orientation='vertical')

# add contour lines
#ax.contour(imSum2, levels = [1, 20, 40,60], colors = 'k')
cbar.ax.set_yticklabels([str(ii / 100.0) for ii in range(10, 100, 20)])
# plt.xlim([600, 1400])
# plt.ylim([ 1700, 700])

ax.axis('off')

ax.set_xlim([600, 1900])
ax.set_ylim([ 1800, 700])

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
#ax.scatter(x = intPoints[:,0], y = intPoints[:,1])

plt.show()

plt.savefig('Figures/SingleAntherHeatmap_magmaCmap_noBlur.png', dpi = 500)

In [162]:
plt.clf()




stackedImg = np.ones((2000, 2000)) * 0.0

for ii in range(len(photoList)):
    # read in image
    
    fpth = photoList[ii]
    img = cv2.imread(fpth,0)
    img = (img < 255) * 1.0
    if ii == 0:
        img = cv2.flip(img, 1)
    img2 = mask * img
    stackedImg = img2 + stackedImg


imSum2 = stackedImg.copy() / np.max(stackedImg) * 100

plt.close('all')


imNA = imSum2
imNA[imNA <= 0] = np.nan
fig, ax = plt.subplots()
im = ax.imshow(imNA, cmap = colormap_r, interpolation='none')
cax = fig.add_axes([0.78, 0.2, 0.05, 0.6])
cbar = fig.colorbar(im, cax=cax, ticks = range(10, 100, 20),  orientation='vertical')

cbar.ax.set_yticklabels([str(ii / 100.0) for ii in range(10, 100, 20)])
ax.set_xlim([200, 2200])
ax.set_ylim([ 1700, 500])

ax.axis('off')


plt.show()

plt.savefig('Figures/SingleAntherHeatmap_rawData_magmaCmap_noBlur.png', dpi = 500)

In [159]:
# add blur to stacked Images
a = stackedImg.copy()
slice_val = 150
b = a[slice_val:-slice_val, slice_val:-slice_val]


plt.clf()
kernel = np.ones((7,7),np.float32)/25.0

imNA = b
imNA[imNA <= 0] = np.nan
imSum2 = np.array(cv2.filter2D(imNA.copy(),-1,kernel))
imsum3 = imSum2.copy() / np.nanmax(imSum2) * 100

plt.close('all')



fig, ax = plt.subplots()
im = ax.imshow(imsum3, cmap = colormap_r, interpolation='none')
cax = fig.add_axes([0.78, 0.2, 0.05, 0.6])
cbar = fig.colorbar(im, cax=cax, ticks = range(10, 100, 20),  orientation='vertical')

cbar.ax.set_yticklabels([str(ii / 100.0) for ii in range(10, 100, 20)])
ax.set_ylim([ 1600, 500])
ax.set_xlim([200, 2200])
ax.axis('off')


plt.show()

plt.savefig('Figures/SingleAntherHeatmap_rawData_Blur_noresamp.png', dpi = 500)

In [188]:
# make one figure with all trajectories
plt.close("all")
fig, axs = plt.subplots(6,5, figsize=(6.5, 7), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .05, wspace=.05)

axs = axs.ravel()


for ii in range(30):
    # read in image
    
    try:
        fpth = photoList[ii]
        img = cv2.imread(fpth,0)
        img = (img < 255) * 1.0
        if ii == 0:
            img = cv2.flip(img, 1)
            
        axs[ii].imshow(img, cmap = colormap_r)
    except:
        pass
    
    axs[ii].axis('off')
    
fig.savefig('Figures/EachPhoto_Separate.png', dpi = 800, bbox_inches='tight')